In [1]:
import os
import glob
import psycopg2
import pandas as pd
import numpy as np
import datetime
from datetime import datetime
from psycopg2.extensions import register_adapter, AsIs
psycopg2.extensions.register_adapter(np.int64, psycopg2._psycopg.AsIs)
psycopg2.extensions.register_adapter(np.int32, psycopg2._psycopg.AsIs)
# %run -i 'Create_Table_queries.py'
from Create_Table_queries import *
%load_ext sql

In [2]:
def process_youtubedata_file(cur, filepath): 
    """
        This function reads one JSON file and read information of videos and youtuber data and saves into video_data and youtuber_data
        Arguments:
        cur: Database Cursor
        filepath: location of JSON files
        Return: None
    """
    # open JSON file
    df = pd.read_json(filepath)

    # ---------insert youtuber record----------
    # write your code here that reads youtuber data from JSON file and insert it into Youtubers_dim table 
    

    # write your code here
    # I checked before that all json files have shape 1 * 10. so just assigning from 0th row
    youtuber_record = (df.iloc[0][3],df.iloc[0][7],df.iloc[0][5],df.iloc[0][4],df.iloc[0][6])
    cur.execute(Youtubers_table_insert, youtuber_record)

    
    # ---------insert video record--------------
    # write your code here that reads youtube videos data from JSON file and insert it into Videos_dim table 

    # write your code here
    year = df.iloc[0][2].astype(int)
    video_record = (df.iloc[0][9],df.iloc[0][1],df.iloc[0][3],year,df.iloc[0][0])
    cur.execute(Videos_table_insert, video_record)

def convert_TS_to_date(ts):
    dt_object = datetime.fromtimestamp(ts)
    _date = dt_object.date()
    _hr = dt_object.hour
    _day = dt_object.day
    _week_number = dt_object.isocalendar()[1]
    _m = dt_object.month
    _year = dt_object.year
    _weekday= dt_object.weekday()
    return _date,_hr,_day,_week_number,_m, _year, _weekday
def get_startDate(ts):
    dt_object = datetime.fromtimestamp(ts)
    _date = dt_object.date()
    return _date
def get_UserData(df, i):
    record = (df.loc[i]['userId'], df.loc[i]['firstName'], df.loc[i]['lastName'], df.loc[i]['gender'], df.loc[i]['level'])
    return record

def process_log_file(cur, filepath):
    """
        This function reads Log files and reads information of time, user and videoplay data and saves into time, user, videoplay
        Arguments:
        cur: Database Cursor
        filepath: location of Log files
        Return: None
    """

    # open log file
    df = pd.read_json(filepath, lines=True)

    # filter by NextVideo action
    df = df[(df['page'] == 'NextVideo')]
        ## Preprocessing
    df['timestamp'] = (df['ts'] -796) / 1000
    df['timestamp'] = df['timestamp'].astype(int)
    for each in df['timestamp']:
#         print(each)
        startTime , hr, day, week, month,year, wd = convert_TS_to_date(each)
#         print(startTime , hr, day, week, month,year, wd)
        cur.execute(Time_table_insert, (startTime , hr, day, week, month,year, wd))
    
    # convert timestamp column to datetime    
    # insert time data records to Time_dim table
        # write your code here

    #********************************************************************
    # load user table
    # insert user records into Users_dim table
    
    for i in df.index:
        data = get_UserData(df, i)
#         print(data)
        cur.execute(Users_table_insert,data)
    
        # write your code here

     #********************************************************************
    # insert Videoplay records in Videoplay_fact table
#     'auth', 'firstName', 'gender', 'itemInSession', 'lastName', 'length',
#        'level', 'location', 'method', 'page', 'registration', 'sessionId',
#        'status', 'ts', 'userAgent', 'userId', 'video', 'youtuber']

    videoId =0
    query = (""" SELECT v.video_id, y.youtuber_id FROM Videos_dim AS v LEFT OUTER JOIN Youtubers_dim AS y ON y.youtuber_id = v.video_id WHERE v.title =%s AND y.name =%s AND v.duration = %s""")
    
#     v_q = "%sql SELECT * FROM Videoplay_fact WHERE ;"
    for i in df.index:
        cur.execute(query, (df.loc[i]['video'],df.loc[i]['youtuber'], df.loc[i]['length']))
        missing_data = cur.fetchone()
        
        if missing_data:
            v_id, y_id = missing_data
        else:
            v_id, y_id = None, None
        d = get_startDate(df.loc[i]["timestamp"])
#         v_id = ''
#         y_id =''
        v_r = (videoId,d,df.loc[i]['userId'],df.loc[i]['level'],v_id,y_id,df.loc[i]['sessionId'],df.loc[i]['location'],df.loc[i]['userAgent'] )
        videoId+=1
#         print(v_r)
#         cur.execute("INSERT INTO Videoplay_fact (videoplay_id, start_time ,user_id , level , video_id ,youtuber_id , session_id , location , user_agent) VALUES (%s,%s, %s,%s,%s, %s,%s,%s,%s)", v_r)
        cur.execute(Videoplay_table_insert, v_r)
        # write your code here

def get_AllFilesPath(filepath):
    fileNames = []
    for root, dirs, files in os.walk(filepath):
        files = glob.glob(os.path.join(root, '*.json'))
        for name in files:
            current = os.path.abspath(name)
            fileNames.append(current)
    return fileNames

def process_data(cur, conn, filepath, func):
    """
        This function get all JSON files in given directory by exploring all sub directories, and process all files that were found using the given function.
        Example: if I give it the path to youtube_data directory which resides in data folder of this assignment,
        and func given is process_youtubedata_file it should get all JSON files in this directories and process each file using process_youtubedata_file function. 
        Arguments:
        cur: Database Cursor
        conn: Database
        filepath: location of JSON files
        func: function to process all files in the directory
        Return: None
    """
    allFiles = get_AllFilesPath(filepath)
    if func == process_youtubedata_file:
        for currentFile in allFiles:
            process_youtubedata_file(cur, currentFile)
    elif func == process_log_file:
        for currentFile in allFiles:
            process_log_file(cur, currentFile)

def main():
    youtube_data = r'D:\data\youtube_data'
    log_data = r'D:\data\log_data'
    conn = psycopg2.connect("host=127.0.0.1 dbname=youtubedb user=postgres password=12345678")
    cur = conn.cursor()
    conn.set_session(autocommit=True)

    process_data(cur, conn, filepath=youtube_data, func=process_youtubedata_file)
    process_data(cur, conn, filepath=log_data, func=process_log_file)

    conn.close()


if __name__ == "__main__":
    main()